In [1]:
import pandas  as pd
import numpy as np
import matplotlib.pylab as plt
import seaborn as sns
import scipy.stats as personr

In [2]:
import pyodbc
print(pyodbc.drivers())

['SQL Server', 'ODBC Driver 18 for SQL Server', 'Microsoft Access Driver (*.mdb, *.accdb)', 'Microsoft Excel Driver (*.xls, *.xlsx, *.xlsm, *.xlsb)', 'Microsoft Access Text Driver (*.txt, *.csv)', 'Microsoft Access dBASE Driver (*.dbf, *.ndx, *.mdx)']


In [17]:
import socket

try:
    s = socket.create_connection(("192.168.137.5", 1433), timeout=3)
    print("✅ SQL Server is reachable!")
except Exception as e:
    print("❌ Connection failed:", e)

✅ SQL Server is reachable!


In [18]:
import pandas as pd
from sqlalchemy import create_engine

username = 'ccna'
password = 'Hamed/.1234'
server_ip = '192.168.137.5'
database = 'TelcoData'

conn_str = (
    f"mssql+pyodbc://{username}:{password}"
    f"@{server_ip}/{database}?driver=ODBC+Driver+18+for+SQL+Server&Encrypt=no"
)

engine = create_engine(conn_str)
# df = pd.read_sql("SELECT TOP 100 * FROM [WA_Fn-UseC_-Telco-Customer-Churn]", engine,)
df = pd.read_sql("SELECT * FROM [WA_Fn-UseC_-Telco-Customer-Churn]", engine,)
df

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.3,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.7,151.65,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7039,2234-XADUH,Female,0,Yes,Yes,72,Yes,Yes,Fiber optic,No,...,Yes,No,Yes,Yes,One year,Yes,Credit card (automatic),103.2,7362.9,No
7040,4801-JZAZL,Female,0,Yes,Yes,11,No,No phone service,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.6,346.45,No
7041,8361-LTMKD,Male,1,Yes,No,4,Yes,Yes,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Mailed check,74.4,306.6,Yes
7042,3186-AJIEK,Male,0,No,No,66,Yes,No,Fiber optic,Yes,...,Yes,Yes,Yes,Yes,Two year,Yes,Bank transfer (automatic),105.65,6844.5,No


In [19]:
query = "SELECT TOP 2 * FROM [WA_Fn-UseC_-Telco-Customer-Churn];"
df = pd.read_sql(query, engine)
df.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No


In [20]:
query = """
SELECT 
    Contract,
    COUNT(*) AS total_customers,
    SUM(CASE WHEN Churn = 'Yes' THEN 1 ELSE 0 END) AS churned_customers,
    ROUND(SUM(CASE WHEN Churn = 'Yes' THEN 1 ELSE 0 END)*100.0 / COUNT(*), 2) AS churn_rate_percent
FROM [WA_Fn-UseC_-Telco-Customer-Churn]
GROUP BY Contract;
"""
df = pd.read_sql_query(query, engine)
df

,Contract,total_customers,churned_customers,churn_rate_percent
0,Month-to-month,3875,1655,42.71
1,Two year,1695,48,2.83
2,One year,1473,166,11.27
3,Contract,1,0,0.00


In [21]:
data = {
    'CustomerID': [1, 2, 3, 4, 5, 6, 7],
    'Contract': ['Monthly', 'One year', 'Monthly', 'Two year', 'Monthly', 'One year', 'Monthly'],
    'churn': ['Yes', 'No', 'Yes', 'No', 'No', 'No', 'Yes']
}



In [22]:
import pandas as pd
df1 = pd.DataFrame(data)
df1

,CustomerID,Contract,churn
0,1,Monthly,Yes
1,2,One year,No
2,3,Monthly,Yes
3,4,Two year,No
4,5,Monthly,No
5,6,One year,No
6,7,Monthly,Yes


In [23]:
results = df1.groupby('Contract').agg(
    total_customers = ('Contract','count'),
    churned_customers = ('churn',lambda x: (x == 'Yes').sum())
)

results['Churn_rate_percent'] = (results['churned_customers'] / results ['total_customers'] * 100).round(2)

In [24]:
print(results.columns)

Index(['total_customers', 'churned_customers', 'Churn_rate_percent'], dtype='object')


In [25]:
results

,total_customers,churned_customers,Churn_rate_percent
Contract,,,
Monthly,4,3,75.0
One year,2,0,0.0
Two year,1,0,0.0


In [31]:
query = """
SELECT Contract, COUNT(*) AS total_customers
FROM Customers
GROUP BY Contract;
"""
df = pd.read_sql_query(query, engine)

In [32]:
df = pd.read_sql_query(query, engine)

ProgrammingError: (pyodbc.ProgrammingError) ('42S02', "[42S02] [Microsoft][ODBC Driver 18 for SQL Server][SQL Server]Invalid object name 'Customers'. (208) (SQLExecDirectW)")
[SQL: 
SELECT Contract, COUNT(*) AS total_customers
FROM Customers
GROUP BY Contract;
]
(Background on this error at: https://sqlalche.me/e/20/f405)

In [33]:
print(engine.table_names())

AttributeError: 'Engine' object has no attribute 'table_names'